<a href="https://colab.research.google.com/github/fabiomatricardi/MetadataIsAllYouNeed/blob/main/FullEDA_DreateDOCS_metatags_userInput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Pickle export with full EDA and KeyBERT

yet to be prepared a new function to stack older data into new

a new function to join more than one CHROMA DB

### Done
- Summarization with metadata
- QnA export in Pickle
- Chunks (350 and 10 overlap) with metadata and Pickle export

In [ ]:
%%capture
!pip install transformers
!pip install langchain
!pip install rich
!pip install gradio
!pip install keybert
!pip install tiktoken

In [ ]:
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=110)
from transformers import pipeline
import gradio as gr
import os
from keybert import KeyBERT

In [ ]:
modelfile = "LaMini-Flan-T5-77M"
repetitionpenalty = 1.3
temperature = 0.3
contextlength=512
logfile = 'DebugLog.txt'
filename = 'Lamini_PDF_fullEDA_keys.txt'

## Test  MBZUAI/LaMini-Flan-T5-77M
with console.status("Loading LaMini77M...",spinner="dots12"):
    model77 = pipeline('text2text-generation',model="MBZUAI/LaMini-Flan-T5-77M")

Output()

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT(model='multi-qa-MiniLM-L6-cos-v1')

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# Load Embeddings for MultiLanguage
from langchain.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-base')

.gitattributes:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [ ]:
logfile = 'KeyBERT-FullEDA.txt'

def writehistory(text):
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(text)
        f.write('\n')
    f.close()

In [ ]:
#########################################################################################
#########    EXTRACT THE KEYWORDS FROM A TEXT, given nGram and Diversity     ############
#########    Return a LIST of tags                                           ############
#########################################################################################
def extract_keys(text, ngram,dvsity):
    import datetime
    import random
    a = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, ngram), stop_words='english',
                              use_mmr=True, diversity=dvsity, )     #highlight=True
    tags = []
    for kw in a:
        tags.append(str(kw[0]))
    timestamped = datetime.datetime.now()
    #LOG THE TEXT AND THE METATAGS
    logging_text = f"LOGGED ON: {str(timestamped)}\nMETADATA: {str(tags)}\nsettings: keyphrase_ngram_range (1,{str(ngram)})  Diversity {str(dvsity)}\n---\nORIGINAL TEXT:\n{text}\n---\n\n"
    writehistory(logging_text)
    return tags

In [ ]:
def SMS(fulltext, llmpipeline, chunks, overlap, documentfilename, title, author,docurl):
  """
  SUS aka Summarize Metadata Suggest
  documentfilename -> str,  'extracted from fileupload',
  title -> str,  title of the document 'input in the GUI',
  author -> str,  author of the document 'input from the GUI',
  docurl -> str,  url of the document 'if any from GUI anyway',
  ---
  Description
  Function that take a long string text and summarize it: returning also
  suggested QnA from the process
  The fulltext is split in Tokens according to chunks and overlap specified
  inputs:
  llmpipeline -> a transformers pipeline instance for the text generation
  fulltext - > string
  chunks, overlap - > integers
  ---
  returns:
  # sum_db constains all original file metadata and the ones from GUI user input
  # qna is a list of dict with EDA in dict format {'question':item,'answer': res}
  # keys a list of ready for LangChain.Document creation with metadata
  """
  from langchain.document_loaders import TextLoader
  from langchain.text_splitter import TokenTextSplitter
  TOKENtext_splitter = TokenTextSplitter(chunk_size=chunks, chunk_overlap=overlap)
  sum_context = TOKENtext_splitter.split_text(fulltext) #create a list
  model77 = llmpipeline
  final = ''
  keys = []
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_bullets = f'''ARTICLE: {text}

    What is a one-paragraph summary of the above article?

    '''
    res = model77(template_bullets, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    final = final + ' '+ res+'\n'
    keys.append({'document' : documentfilename,
                'title' : title,
                'author' : author,
                'url' : docurl,
                'doc': text,
                'keywords' : extract_keys(text, 1, 0.34)
    })
  ## REMOVED REWRITING FNCTION ####
  tags_summary = extract_keys(final, 1, 0.34)
  elaps = datetime.datetime.now() - strt
  console.print(Markdown("## SUMMARY"))
  console.print(Markdown(final))
  console.print(Markdown("### METATAGS"))
  console.print(Markdown(str(tags_summary)))
  console.print("[green2 bold]---")
  console.print(f"[red1 bold]Full RAG PREPROCESSING Completed in {elaps}")
  console.print(Markdown("---"))
  logger = "# SUMMARY and METATAGS\n---\n#SUMMARY\n" + final + "#MetaTAGS: "+ str(tags_summary) +"\n---\n\n"
  writehistory(logger)

  # Generate Suggested questions from the text
  # Then Reply to the questions
  console.print(f"[green2 bold]Generating Qna...")
  finalqna = ''
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_final = f'''{text}.\nAsk few question about this article.
  '''
    res = model77(template_final, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    finalqna = finalqna + '\n '+ res

  delt = datetime.datetime.now()-strt
  console.print(Markdown("---"))
  console.print(f"[red1 bold]Questions generated in {delt}")
  lst = finalqna.split('\n')
  final_lst = []
  for items in lst:
    if items == '':
      pass
    else:
      final_lst.append(items)

  qna = []
  for item in final_lst:
    question = item

    template_qna = f'''Read this and answer the question. If the question is unanswerable, ""say \"unanswerable\".\n\n{final}\n\n{question}
    '''

    start = datetime.datetime.now()
    res = model77(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    elaps = datetime.datetime.now() - start
    """
    console.print(f"[bold deep_sky_blue1]{question}")
    console.print(Markdown(res))
    console.print(f"[red1 bold]Qna Completed in {elaps}")
    console.print(Markdown("---"))
    """
    qna.append({'question':item,
                'answer': res})
    logger = "QUESTION: " + question + "\nANSWER: "+ res + "\n---\n\n"
    writehistory(logger)
  sum_db = {'document' : documentfilename,
            'title' : title,
            'author' : author,
            'url' : docurl,
            'summary' : final,
            'keywords' : tags_summary,
             'qna' : qna}
  # sum_db constains all original file metadata and the ones from GUI user input
  # qna is a list of dict with EDA questions and answers
  # keys a list of ready for LangChain.Document creation with metadata
  return sum_db, qna, keys

#Funtion to club the ones above

def start_SMS(fulltext, documentfilename, title, author,url):
  console.print("Starting TXT Summarization...")
  global SUMfinal
  global QNASet
  global DOCSkeys
  SUMfinal,QNASet, DOCSkeys = SMS(fulltext, model77, 450, 10, documentfilename, title, author,url)
  console.print("Process Completed!")
  return SUMfinal

def printQNA(produced_qna):
  head = """### Generated Qna<br><br>"""
  qnaset = " "
  for item in produced_qna:
    temp = f"""
> **Question: {item['question']}**<br>
> *Answer: {item['answer']}*<br>
"""
    qnaset = qnaset + temp
  finalQnA = head + qnaset
  console.print(Markdown(finalQnA))
  return finalQnA

def start_search(b):
  global fulltext
  with open(b, encoding="utf8") as f:
    fulltext = f.read()
  f.close()
  console.print("[bold deep_sky_blue1]text has been captured")
  console.print(Markdown("---"))
  console.print("Saved in variable `fulltext`")
  statmessage = "Operation Successful\nClick on Start Summarization ↗️"
  return fulltext

def generate(instruction):
  prompt = instruction
  start = datetime.datetime.now()
  with console.status("AI is thinking...",spinner="dots12"):
    output = model77(prompt, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  delta = datetime.datetime.now()-start
  console.print(f"[green1 italic]:two_o’clock: generated in {delta}")
  console.print(f"[bold bright_yellow]🦙 LaMini-Flan-77M: {output}")
  return output, delta

In [ ]:
fulltext = start_search('/content/2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt')

text has been captured

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Saved in variable `fulltext`

In [ ]:
SUMfinal,QNASet, DOCSkeys = SMS(fulltext, model77, 350, 10,
                                '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
                                'Governing societies with Artificial Intelligence',
                                'Giles Crouch','https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473')
console.print("Process Completed!")

Output()

SUMMARY

The article discusses how the use of AI in society and social governance is a major concern. It highlights the
importance of culture, economic systems, family systems, politics, militaries, art, architecture, literature, 
and human creativity. The article discusses the impact of AI on society and its complexity, including the     
increasing complexity of systems of governance and sociocultural systems. It also mentions that there are     
risks associated with co-evolved technologies to run them. The article discusses the importance of AI in      
managing complexity, addressing issues with critical thinking and systems thinking. It also mentions that     
technology can be used to help govern societies by providing solutions for complex problems, but it does not  
necessarily mean that all technologies are neutral or unintended consequences. The article explains how to    
foster true social innovations in society by setting regulations and laws that don't stifle AI, while also    
providing insights into race and gender biases. It suggests that tech leaders should take an approach towards 
making AI more accessible to the public through models and ways that enable it to evolve. The article         
discusses how AI can help govern societies, but it does not necessarily mean that social enlightenment will   
come from humans. It suggests that the only solution to this problem is to raise consciousness as some propose
is inevitable and move ahead.

METATAGS

['ai', 'article', 'society', 'enlightenment', 'importance']

...

Generating Qna...

Output()

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Questions generated in 0:00:06.570573

Process Completed!

In [ ]:
questans = printQNA(QNASet)

Generated Qna                                                 



▌ Question:  What is the role of social governance in shaping society and AI? Answer: The role of social    
▌ governance in shaping society and AI is to address issues with critical thinking and systems thinking,    
▌ provide solutions for complex problems, foster true social innovations by setting regulations and laws    
▌ that don't stifle AI, and provide insights into race and gender biases.                                   



▌ Question:  What is the reason for a lack of complexity in AI tools and how does it affect society? Answer:
▌ The reason for a lack of complexity in AI tools is the increasing complexity of systems of governance and 
▌ sociocultural systems. It affects society by setting regulations and laws that don't stifle AI, while also
▌ providing insights into race and gender biases.                                                           



▌ Question:  What are the three principles that should be etched in stone and why do they differ from each  
▌ other? Answer: The three principles that should be etched in stone are culture, economic systems, family  
▌ systems, politics, militaries, art, architecture, literature, and human creativity. They differ from each 
▌ other because they emphasize the importance of AI in managing complexity, providing solutions for complex 
▌ problems, and raising consciousness as some propose is inevitable and move ahead.                         



▌ Question:  What is the main argument in this article about technology adoption? Answer: The main argument 
▌ in this article about technology adoption is that it can help govern societies by providing solutions for 
▌ complex problems, but it does not necessarily mean that all technologies are neutral or unintended        
▌ consequences.                                                                                             



▌ Question:  What is the significance of AI in society and how does it impact social enlightenment? Answer: 
▌ The significance of AI in society is the increasing complexity of systems of governance and sociocultural 
▌ systems, as well as risks associated with co-evolved technologies to run them. It can help govern         
▌ societies by providing solutions for complex problems, but it does not necessarily mean that all          
▌ technologies are neutral or unintended consequences.                                                      

In [ ]:
console.print(SUMfinal)

In [ ]:
console.print(DOCSkeys)

In [ ]:
############### CREATE SUMMARY DOC DATABASE ##################
"""
FROM SUMfinal
    'document': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
    'title': 'Governing societies with Artificial Intelligence',
    'author': 'Giles Crouch',
    'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
    'summary': " ...",
    'keywords': ['ai', 'article', 'society', 'enlightenment', 'importance'],
"""
from langchain.schema.document import Document
docsum = Document(page_content = SUMfinal['summary'],
                  metadata = {'source': SUMfinal['document'],
                              'type': 'summary',
                              'title': SUMfinal['title'],
                              'author': SUMfinal['author'],
                              'url' : SUMfinal['url'],
                              'keywords' : SUMfinal['keywords']
                              })

In [ ]:
############### CREATE cHUnKS DOC DATABASE ##################
"""
FROM DOCSkeys
{
        'document': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
        'title': 'Governing societies with Artificial Intelligence',
        'author': 'Giles Crouch',
        'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
        'doc': ' as inherently true, ..ought that through beyond the narrow',
        'keywords': ['ai', 'innovation', 'societal', 'industry', 'slowing']
    }

"""
from langchain.schema.document import Document
docdocs = []
for i in range(0,len(DOCSkeys)):
  docdocs.append(Document(page_content = DOCSkeys[i]['doc'],
                          metadata = {'source': DOCSkeys[i]['document'],
                              'type': 'chunk',
                              'title': DOCSkeys[i]['title'],
                              'author': DOCSkeys[i]['author'],
                              'url' : DOCSkeys[i]['url'],
                              'keywords' : DOCSkeys[i]['keywords']
                              }))

In [ ]:
console.print(docdocs)

### Save and Load DATASET to and from PICKLE file

In [ ]:
## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
output = open('DocumentData.pkl', 'wb')
pickle.dump(docdocs, output)
output.close()
console.print(Markdown("> Documents Data saved..."))
console.print(" - ")
## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
output = open('summaryData.pkl', 'wb')
pickle.dump(docsum, output)
output.close()
console.print(Markdown("> Summary Data saved..."))

▌ Documents Data saved...                                                                                   

-

▌ Summary Data saved...                                                                                     

In [ ]:
def SaveFirstData(datafilename,datadocs, summaryfilename, sumdocs):
  import pickle
  ## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
  output = open(datafilename, 'wb')
  pickle.dump(datadocs, output)
  output.close()
  console.print(Markdown(f"> Documents Data saved in {datafilename}..."))
  console.print(" - ")
  ## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
  output = open(summaryfilename, 'wb')
  pickle.dump(sumdocs, output)
  output.close()
  console.print(Markdown("> Summary Data saved in {summaryfilename}..."))

▌ Documents Data saved...                                                                                   

▌ Summary Data saved...                                                                                     

In [ ]:
import pickle
##LOAD DOCS PICKLE
pkl_file = open('/content/DocumentData.pkl', 'rb')
data_docs = pickle.load(pkl_file)
pkl_file.close()
console.print("Documents Loaded from Pickle")
console.print(Markdown("---"))
console.print(data_docs)
console.print(Markdown("---"))



Documents Loaded from Pickle

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

[
    Document(
        page_content='Governing societies with Artificial Intelligence | 
Medium\nhttps://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473\nTITLE: 
Governing Society and Artificial Intelligence\nAUTHOR: Giles Crouch | Digital Anthropologist\n\nMuch of the 
hype and chatter around Artificial Intelligence today is how it will impact the workplace, the entertainment 
industry, human creativity and productivity. Increasingly, there are debates on ethics, human rights and human
agency. These are important. But what about how we govern our societies with AI?\nSocial governance is an 
element of culture which also includes economic systems, family systems, politics, militaries, art, 
architecture and literature. Sometimes, such as in a democracy, we have a say in these systems. In 
dictatorships, we do not.\nThe mechanism by which we create and operate our social governance systems is 
bureaucracy. Which were well honed by corporations and go back many thousands of years. One might suggest that
bureaucracy was humanity’s first multiplayer game.\nHumans have been fiddling about with all kinds of 
different governance systems for thousands of years, including hunter-gatherer and foraging societies. When we
collectively decide we’re no longer thrilled with whatever system, we either run away and create something 
else or we have revolutions and whoever wins creates something new. The one constant? Bureaucracies. Rinse, 
wash, repeat.\nWith the advent of Generative AI (GAI) through Large Language Models (LLMs) like ChatGPT, 
humanity finally had an opportunity to play with AI in a way it hadn’t before. Prior to GAI, our broader 
social understanding',
        metadata={
            'source': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
            'type': 'chunk',
            'title': 'Governing societies with Artificial Intelligence',
            'author': 'Giles Crouch',
            'url': 
'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
            'keywords': ['society', 'governance', 'ai', 'governing', 'humanity']
        }
    ),
    Document(
        page_content='. Prior to GAI, our broader social understanding of AI was via Hollywood and 
Science-Fiction and it was very often rather dystopian.\nNews articles today tend to either declare the end of
the world is nigh or utopia will be in the earnings reports of Q4 of 2024. As the news media has often said, 
“if it bleeds, it leads” (a.k.a. clickbait today.) Unfortunately, just as so much else today is reduced to 
this or that arguments, the same has happened to AI.\nThe reality is more likely to be somewhere in the middle
when it comes to AI tools. Likely edging towards more benefits than harms.\nSo we’ve not really been set up to
receive AI in a nice way. It should be of little surprise in the launch and hype of GAI that we collectively 
panicked and ran about the backyard flailing our arms and screaming that the sky is falling. The prophets of 
AI in Silicon Valley didn’t much help with proclamations of the arrival of a new god in town and here’s your 
new religion; the new religious text is Large Language Models.\nYet there is a very strong case for making AI 
work for us in governing our current and future societies. The increasing complexity of our systems of 
governance along with our sociocultural systems. Human brains, working collectively, are quite a powerful 
thing. But our collective cognitive abilities only go so far. There are risks such a Group Think, among 
others.\nWe have co-evolved technologies because we need them to survive. As our societies and the 
technologies we use to run them have evolved, so has complexity. There’s a',
        metadata={
            'source': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
            'type': 'chunk',
            'title': 'Governing societies with Artificial Intellige

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
import pickle
##LOAD SUMS PICKLE
pkl_file = open('/content/summaryData.pkl', 'rb')
sums_docs = pickle.load(pkl_file)
pkl_file.close()
console.print("Summaries Loaded from Pickle")
console.print(Markdown("---"))
console.print(sums_docs)
console.print(Markdown("---"))

Summaries Loaded from Pickle

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Document(
    page_content=" The article discusses how the use of AI in society and social governance is a major 
concern. It highlights the importance of culture, economic systems, family systems, politics, militaries, art,
architecture, literature, and human creativity.\n The article discusses the impact of AI on society and its 
complexity, including the increasing complexity of systems of governance and sociocultural systems. It also 
mentions that there are risks associated with co-evolved technologies to run them.\n The article discusses the
importance of AI in managing complexity, addressing issues with critical thinking and systems thinking. It 
also mentions that technology can be used to help govern societies by providing solutions for complex 
problems, but it does not necessarily mean that all technologies are neutral or unintended consequences.\n The
article explains how to foster true social innovations in society by setting regulations and laws that don't 
stifle AI, while also providing insights into race and gender biases. It suggests that tech leaders should 
take an approach towards making AI more accessible to the public through models and ways that enable it to 
evolve.\n The article discusses how AI can help govern societies, but it does not necessarily mean that social
enlightenment will come from humans. It suggests that the only solution to this problem is to raise 
consciousness as some propose is inevitable and move ahead.\n",
    metadata={
        'source': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
        'type': 'summary',
        'title': 'Governing societies with Artificial Intelligence',
        'author': 'Giles Crouch',
        'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
        'keywords': ['ai', 'article', 'society', 'enlightenment', 'importance']
    }
)

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

### yet to be prepared a new function to stack older data into new

### a new function to join more than one CHROMA DB

### Create LangChain Documents and Vector Store index

In [ ]:
#Chuncking at 200 tokens no overlap
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=0)
sum_context = TOKENtext_splitter.split_text(fulltext) #create a list

In [ ]:
def createDB(docs, embeddings, dbname):
  """
  Function that create a Chroma Vector store of splitted documents
  with provided embeddings, and save it locally.
  docs: text_splitter docs object
  embeddings: HuggingFace embeddings object
  dbname: string, the name of the persistent db
  RETURN  the Chroma db object
  """
  from langchain.vectorstores import Chroma
  import datetime
  start = datetime.datetime.now()
  db = Chroma.from_documents(docs, embeddings,persist_directory=f"./{dbname}")
  stop = datetime.datetime.now()
  delta = stop-start
  print(f"Vector db generated in {delta}")
  return db


In [ ]:
db3 = createDB(docdocs, hf_embeddings, 'db-200-0MULTI')

Vector db generated in 0:00:01.943620

```
# load from disk
db3 = Chroma(persist_directory="./chroma_db480tok-20", embedding_function=hf_embeddings)
```



In [ ]:
dataset = CreateRAGAS_DF(QNASet,CONTEXTSet,SUMfinal,llm)

Output()

In [ ]:
console.print(dataset)

### Save and Load DATASET to and from PICKLE file

In [ ]:
import pickle

output = open('dataset.pkl', 'wb')
pickle.dump(dataset, output)

output.close()

In [ ]:
import pprint, pickle

pkl_file = open('dataset.pkl', 'rb')

data2 = pickle.load(pkl_file)
pkl_file.close()
console.print(data2)